In [28]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer
from plot_learning_curves import plot_learning_curves

In [29]:
X_train_path= os.path.join("..", "data","input", "X_train_Wwou3IE.csv")
X_train_preprocessed = pd.read_csv(X_train_path, delimiter=',')
X_test_path= os.path.join("..", "data","input", "X_test_GgyECq8.csv")
X_test_preprocessed = pd.read_csv(X_test_path, delimiter=',')
y_train_path= os.path.join("..", "data","input", "y_train_jJtXgMX.csv")
y_train_preprocessed = pd.read_csv(y_train_path, delimiter=',')

delivery_start = X_test_preprocessed["DELIVERY_START"]

cold_rate = {
    1: 100,  # January
    2: 90,   # February
    3: 70,   # March
    4: 50,   # April
    5: 30,   # May
    6: 10,   # June
    7: 0,    # July
    8: 5,    # August
    9: 20,   # September
    10: 40,  # October
    11: 60,  # November
    12: 80   # December
}

X_train_preprocessed['DELIVERY_START'] = pd.to_datetime(X_train_preprocessed['DELIVERY_START'], utc = True)
X_test_preprocessed['DELIVERY_START'] = pd.to_datetime(X_test_preprocessed['DELIVERY_START'], utc = True)

X_train_preprocessed['month'] = X_train_preprocessed['DELIVERY_START'].dt.month
X_train_preprocessed['cold_rate'] = X_train_preprocessed['month'].map(cold_rate)
X_train_preprocessed.drop('month', axis=1, inplace=True)
X_test_preprocessed['month'] = X_test_preprocessed['DELIVERY_START'].dt.month
X_test_preprocessed['cold_rate'] = X_test_preprocessed['month'].map(cold_rate)
X_test_preprocessed.drop('month', axis=1, inplace=True)

In [30]:
X_train = process_data(X_train_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_test_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(y_train_preprocessed.copy(deep=True), None, None, None)
y_train['spot_id_delta'] = y_train['spot_id_delta'].apply(lambda x: -1 if x < 0 else 1)

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Initialize the KNN model
# You can adjust 'n_neighbors' based on your validation tests to find the best value
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model on your training data
knn.fit(X_train, y_train)

# Predict on your test data
y_pred = knn.predict(X_test)

d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_test_knn1.csv", index = False)
y_pred.shape

c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


(4942, 2)

Accuracy around : 0.6723243752946724 <> score of 0,488